Deep Learning
=============

Assignment 5
------------

The goal of this assignment is to train a skip-gram model over [Text8](http://mattmahoney.net/dc/textdata) data.

In [13]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import collections
import math
import numpy as np
import os
import random
import tensorflow as tf
import zipfile
from matplotlib import pylab
from six.moves import range
from six.moves.urllib.request import urlretrieve
from sklearn.manifold import TSNE

Download the data from the source website if necessary.

In [14]:
url = 'http://mattmahoney.net/dc/'

def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified %s' % filename)
  else:
    print(statinfo.st_size)
    raise Exception(
      'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

filename = maybe_download('text8.zip', 31344016)

Found and verified text8.zip


Read the data into a string.

In [15]:
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return tf.compat.as_str(f.read(name)).split()
  f.close()
  
words = read_data(filename)
print('Data size %d' % len(words))

Data size 17005207


Build the dictionary and replace rare words with UNK token.

In [16]:
vocabulary_size = 50000

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys())) 
  return data, count, dictionary, reverse_dictionary

data, count, dictionary, reverse_dictionary = build_dataset(words)
print('Most common words (+UNK)', count[:5])
print('Sample data', data[:10])
del words  # Hint to reduce memory.

Most common words (+UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156]


In [17]:
print(type(data), len(data)) # 'word index' of each word in the entire 'words' data. the 'word index' is defined in 'dictionary' 
print(type(count), len(count)) # contains words and their counts, i.e., ('of', 593677).
print(type(dictionary), len(dictionary)) # use 'word' to find its 'ranked index'
print(type(reverse_dictionary), len(reverse_dictionary)) # use 'ranked index' to find its 'word'

word_text = 'of'
print('word: [%s]-> '%word_text, dictionary[word_text])
word_text = 2
print('word: [%s]-> '%word_text, reverse_dictionary[word_text])

for word_idx in range(10):
    print('idx:', word_idx, '->', data[word_idx], count[word_idx])


<type 'list'> 17005207
<type 'list'> 50000
<type 'dict'> 50000
<type 'dict'> 50000
word: [of]->  2
word: [2]->  of
idx: 0 -> 5239 ['UNK', 418391]
idx: 1 -> 3084 ('the', 1061396)
idx: 2 -> 12 ('of', 593677)
idx: 3 -> 6 ('and', 416629)
idx: 4 -> 195 ('one', 411764)
idx: 5 -> 2 ('in', 372201)
idx: 6 -> 3137 ('a', 325873)
idx: 7 -> 46 ('to', 316376)
idx: 8 -> 59 ('zero', 264975)
idx: 9 -> 156 ('nine', 250430)


Function to generate a training batch for the skip-gram model.

In [18]:
data_index = 0

# batch: center `word`
# labels: `label[a]` is the neighboring word around the center word `batch[a]`

def generate_batch(batch_size, num_skips, skip_window):
    # batch_size: one independent segment. All the processes are in this batch. 
    # num_skips: it is the number of neighbors around the center word. (batch_size % num_skips == 0)
    # skip_window: it's the single side size of the window `span`. a window (`span`) of size 3 is of the `skip_window`=1 (batch_size % num_skips == 0)

  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
#   print('buffer:', type(buffer), buffer)
#   print('span:', type(span), span)
  
#   print('[PART I]:')
  # get buffer, which stores the `word index` of the words in the window of size `span`, the left most position of this window is `data_index`
  for _ in range(span):
#     print(data_index, data[data_index])
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data) # repeat
#   print('buffer_start:', type(buffer), buffer,'\n') # 'word index': with the 'data_index' in entire words data, it stores the 'word index' defined in dictionary 
  
#   print('[PART II]:')
#   batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  batch = -1*np.ones(shape=(batch_size), dtype=np.int32)
#   print('batch(-1,-1):', type(batch), batch)
#   labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)  
  labels = -1*np.ones(shape=(batch_size, 1), dtype=np.int32)
#   print('labels(-1,-1):', type(labels), np.transpose(labels))
#   print(batch_size, num_skips, batch_size // num_skips)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer, range [0,span-1]
#     print('target:', type(target), target)
    targets_to_avoid = [ skip_window ] # used target
#     print('targets_to_avoid:', type(targets_to_avoid), targets_to_avoid)
    
    for j in range(num_skips):
#       print('skip_window:',skip_window)
      while target in targets_to_avoid: # select another `target`
        target = random.randint(0, span - 1) # random select from the current `span` size
      targets_to_avoid.append(target) # add new `target` to list
#       print('targets_to_avoid:', type(targets_to_avoid), targets_to_avoid)
      batch[i * num_skips + j] = buffer[skip_window] # 'word index' of the center word in 'buffer'
#       print('batch(%d,%d):'%(i,j), type(batch), batch)
      labels[i * num_skips + j, 0] = buffer[target] # `word index` of the `target`
#       print('labels(%d,%d):'%(i,j), type(labels), np.transpose(labels))
    
    buffer.append(data[data_index])
#     print('buffer_end:', type(buffer), buffer,'\n')
    data_index = (data_index + 1) % len(data)
  return batch, labels

print('data:', [reverse_dictionary[di] for di in data[:8]])

for num_skips, skip_window in [(2, 1), (4, 2)]:
    print('\nwith num_skips = %d and skip_window = %d:' % (num_skips, skip_window))
    data_index = 0
    batch, labels = generate_batch(batch_size=8, num_skips=num_skips, skip_window=skip_window)
    print('    batch:', [reverse_dictionary[bi] for bi in batch])
    print('    labels:', [reverse_dictionary[li] for li in labels.reshape(8)])

data: ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

with num_skips = 2 and skip_window = 1:
    batch: ['originated', 'originated', 'as', 'as', 'a', 'a', 'term', 'term']
    labels: ['anarchism', 'as', 'a', 'originated', 'as', 'term', 'of', 'a']

with num_skips = 4 and skip_window = 2:
    batch: ['as', 'as', 'as', 'as', 'a', 'a', 'a', 'a']
    labels: ['originated', 'term', 'a', 'anarchism', 'term', 'of', 'originated', 'as']


Train a skip-gram model.

In [7]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    print('train_dataset:', train_dataset.get_shape())
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    print('train_labels:', train_labels.get_shape())
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    print('valid_dataset:', valid_dataset.get_shape())

    # Variables.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    print('embeddings:', embeddings.get_shape())
    softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],stddev=1.0 / math.sqrt(embedding_size)))
    print('softmax_weights:', softmax_weights.get_shape())
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    print('softmax_biases:', softmax_biases.get_shape())

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    print('embed:', embed.get_shape())
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,train_labels, num_sampled, vocabulary_size))
    print('loss:', loss.get_shape())

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
 
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    print('norm:', norm.get_shape())
    normalized_embeddings = embeddings / norm # use normed embedding for the following inner product
    print('normalized_embeddings:', normalized_embeddings.get_shape())
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset) # get the embeddings of the words in valid set
    print('valid_embeddings:', valid_embeddings.get_shape())
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings)) # calc the distance, each value denotes the similarity between two words
    print('similarity:', similarity.get_shape())
    # so in this case, the weights and bias are not needed for the appliation. The only thing it needs is just the embeddings. 

train_dataset: (128,)
train_labels: (128, 1)
valid_dataset: (16,)
embeddings: (50000, 128)
softmax_weights: (50000, 128)
softmax_biases: (50000,)
embed: (128, 128)
loss: ()
norm: (50000, 1)
normalized_embeddings: (50000, 128)
valid_embeddings: (16, 128)
similarity: (16, 50000)


In [8]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 8.098579
Nearest to his: baresi, distal, ivb, artistry, acts, docks, easterly, procreation,
Nearest to used: boldness, amperes, alois, katydids, hereditary, vertebrate, retellings, pogues,
Nearest to five: hag, bsd, ulaanbaatar, icmp, brilliance, goddard, quarto, fdp,
Nearest to of: cohabitation, pubescent, discounted, scriptorium, pillars, discordianism, sideline, hale,
Nearest to on: acquitted, interconnects, ramayana, historically, softened, above, cbc, resort,
Nearest to be: mishneh, pretoria, alters, speciality, stances, disillusioned, palahniuk, takeoff,
Nearest to after: molded, zoroastrianism, ulrich, nasional, proceeding, autoroute, tremendous, government,
Nearest to most: hurts, panthers, rested, agincourt, ges, interfere, denotes, corning,
Nearest to however: fresh, arise, respiration, exec, curbed, orwell, yat, flunitrazepam,
Nearest to with: cans, mehmet, blockbusters, vom, demographics, using, optative, lincoln,
Nearest to they: midday,

In [9]:
num_points = 400

tsne = TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000)
two_d_embeddings = tsne.fit_transform(final_embeddings[1:num_points+1, :])

In [10]:
def plot(embeddings, labels):
  assert embeddings.shape[0] >= len(labels), 'More labels than embeddings'
  pylab.figure(figsize=(15,15))  # in inches
  for i, label in enumerate(labels):
    x, y = embeddings[i,:]
    pylab.scatter(x, y)
    pylab.annotate(label, xy=(x, y), xytext=(5, 2), textcoords='offset points',
                   ha='right', va='bottom')
  pylab.show()

words = [reverse_dictionary[i] for i in range(1, num_points+1)]
plot(two_d_embeddings, words)

---

Problem
-------

An alternative to Word2Vec is called [CBOW](http://arxiv.org/abs/1301.3781) (Continuous Bag of Words). In the CBOW model, instead of predicting a context word from a word vector, you predict a word from the sum of all the word vectors in its context. Implement and evaluate a CBOW model trained on the text8 dataset.

---

In [42]:
batch_size = 128
embedding_size = 128 # Dimension of the embedding vector.
skip_window = 1 # How many words to consider left and right.
num_skips = 2 # How many times to reuse an input to generate a label.
# We pick a random validation set to sample nearest neighbors. here we limit the
# validation samples to the words that have a low numeric ID, which by
# construction are also the most frequent. 
valid_size = 16 # Random set of words to evaluate similarity on.
valid_window = 100 # Only pick dev samples in the head of the distribution.
valid_examples = np.array(random.sample(range(valid_window), valid_size))
num_sampled = 64 # Number of negative examples to sample.

graph = tf.Graph()

with graph.as_default():

    # Input data.
    train_dataset = tf.placeholder(tf.int32, shape=[batch_size])
    print('train_dataset:', train_dataset.get_shape())
    train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1])
    print('train_labels:', train_labels.get_shape())
    valid_dataset = tf.constant(valid_examples, dtype=tf.int32)
    print('valid_dataset:', valid_dataset.get_shape())

    # Variables.
    embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
    print('embeddings:', embeddings.get_shape())
    softmax_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size],stddev=1.0 / math.sqrt(embedding_size)))
    print('softmax_weights:', softmax_weights.get_shape())
    softmax_biases = tf.Variable(tf.zeros([vocabulary_size]))
    print('softmax_biases:', softmax_biases.get_shape())

    # Model.
    # Look up embeddings for inputs.
    embed = tf.nn.embedding_lookup(embeddings, train_dataset)
    print('embed:', embed.get_shape())
    # Compute the softmax loss, using a sample of the negative labels each time.
    loss = tf.reduce_mean(tf.nn.sampled_softmax_loss(softmax_weights, softmax_biases, embed,train_labels, num_sampled, vocabulary_size))
    print('loss:', loss.get_shape())

    # Optimizer.
    optimizer = tf.train.AdagradOptimizer(1.0).minimize(loss)
 
    # Compute the similarity between minibatch examples and all embeddings.
    # We use the cosine distance:
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
    print('norm:', norm.get_shape())
    normalized_embeddings = embeddings / norm # use normed embedding for the following inner product
    print('normalized_embeddings:', normalized_embeddings.get_shape())
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset) # get the embeddings of the words in valid set
    print('valid_embeddings:', valid_embeddings.get_shape())
    similarity = tf.matmul(valid_embeddings, tf.transpose(normalized_embeddings)) # calc the distance, each value denotes the similarity between two words
    print('similarity:', similarity.get_shape())
    # so in this case, the weights and bias are not needed for the appliation. The only thing it needs is just the embeddings. 

train_dataset: (128,)
train_labels: (128, 1)
valid_dataset: (16,)
embeddings: (50000, 128)
softmax_weights: (50000, 128)
softmax_biases: (50000,)
embed: (128, 128)
loss: ()
norm: (50000, 1)
normalized_embeddings: (50000, 128)
valid_embeddings: (16, 128)
similarity: (16, 50000)


In [41]:
num_steps = 100001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print('Initialized')
  average_loss = 0
  for step in range(num_steps):
    batch_data, batch_labels = generate_batch(batch_size, num_skips, skip_window)
    feed_dict = {train_dataset : batch_data, train_labels : batch_labels}
    _, l = session.run([optimizer, loss], feed_dict=feed_dict)
    
    average_loss += l
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print('Average loss at step %d: %f' % (step, average_loss))
      average_loss = 0
    
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval()
      for i in xrange(valid_size):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log = 'Nearest to %s:' % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log = '%s %s,' % (log, close_word)
        print(log)
  final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step 0: 7.707431
Nearest to states: helvetii, ave, rhetorician, yes, akiva, grimoire, event, khanates,
Nearest to three: coalition, biomechanics, sympathized, medes, autonomy, augmenting, zander, daniel,
Nearest to more: esquire, koffice, endpoints, reopen, harlequin, fiorentina, shambles, handlebar,
Nearest to this: khaki, ferranti, fingerspelling, barada, amex, aai, gurus, fleury,
Nearest to been: subroutine, seeing, ism, pointing, outdated, mcmillan, murdoc, saronic,
Nearest to other: deceived, reasons, ghent, telecommunications, kenning, minuscule, tutti, bayezid,
Nearest to between: meritorious, bland, remarkably, obi, sem, fath, hunt, ticino,
Nearest to world: aeacus, batsman, modalities, jubal, brewers, upu, arcology, story,
Nearest to on: skepticism, hesitant, lick, live, mischievous, lamont, datatypes, ocular,
Nearest to in: forges, eridanus, setbacks, bernese, tantric, keeps, geezer, niels,
Nearest to which: recycling, mancini, jumaada, swelled, ap

KeyboardInterrupt: 